<a href="https://colab.research.google.com/github/guillemwilly/nn-zero-to-hero/blob/master/B2VocabListMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##This program takes as an imput a book in the form of a text file and produces a list of words ordered by frequency corresponding to the CEFR (A1 to C2) level indicated by the user.


In [123]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 28.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


### here we dont need many packages just spacy and panda as well as io to pass the correct data type to the panda functions

In [124]:
import spacy
import pandas as pd
from io import StringIO

In [125]:
# pipeline
nlp = spacy.load("es_core_news_sm")

## Here we just define a funtion called "scraper" which is just gonna take the file fom the given url and take the data here I also have it cleaning up the opening paragraph of the data I am using

In [126]:
def scraper(url):
 # scrape url
  response = requests.get(url)
  text = response.text
  # delete the opening paragraph

  num_lines_to_delete = 12


  lines = text.split('\n')

  # delete the lines
  clean_text = '\n'.join(lines[num_lines_to_delete:])

  # return
  return clean_text

## Here we call the function that we just defined.

In [ ]:
url1 = "https://www.wordfrequency.info/files/spanish/spanish_lemmas20k.txt"
word_freq = scraper(url)

print(word_freq)

## now we create the panda data frame to make the data easier to compare to in the later algorthithm

In [ ]:
# make a panda data frame

# create data frame
columns = ["ID", "freq", "lemma", "PoS"]
df = pd.read_csv(StringIO(word_freq), sep = '\s+', skiprows = 2, names = columns)

print(df)


##Using the Panda Data frame created we can then look to tokenize a text in our target language which is Spanish here. Here we call the file for the Book we are using and store it in the variable "book_text"

In [ ]:
with open("/content/Book_Collection/book.txt", "r") as file:
  book_text = file.read()

  print(book_text)

##Now I will process the book using Spacy to tokenize it so that it can be worked with easily. Within the function is a lemmatizer and a function to make the word lowercase, this is to avoid cases where case or grammatical morphemes might make the algorithm miss any coincidences.

In [130]:
# Function to filter words in the book based on the frequency
def filter_words(text):
    # Use spacy to create an NLP object
    doc = nlp(text)
    # Extract words with frequency greater than the pre-established threshold and lemmatize
    filtered_words = [token.text.lower() for token in doc if token.lemma_.lower() in df[df['ID'] > 4991]['lemma'].tolist()]
    return filtered_words

filtered_words = filter_words(book_text)

print (filtered_words)


['prólogo', 'mugrientos', 'intersticios', 'o', 'o', 'rellenan', 'según', 'ciudadela', 'chorrean', 'volados', 'estampas', 'moscas', 'labradores', 'carpinteros', 'labradores', 'carpinteros', 'apasionadas', 'ciudadela', 'taberna', 'ciudadela', 'francia', 'labrador', 'capitaneaba', 'bárbaras', 'espontáneamente', 'ladrón', 'ladrones', 'maltrecho', 'o', 'taberna', 'taberna', 'o', 'feo', 'ladrón', 'vagabundo', 'veleidades', 'según', 'ciudadela', 'ciudadela', 'lagartos', 'comarca', 'párroco', 'ladrido', 'péndulo', 'olfateando', 'taberna', 'o', 'vagabundo', 'ciudadela', 'ciudadela', 'taberna', 'mostrador', 'taberna', 'taberna', 'divergencias', 'taberna', 'azuzaba', 'o', 'o', 'graciosa', 'taberna', 'excitado', 'o', 'adjetivo', 'campeaba', 'lobos', 'lobo', 'venerable', 'blasón', 'hondonada', 'según', 'catalina', 'débil', 'evangélica', 'costillas', 'desliz', 'catalina', 'privilegiada', 'catalina', 'o', 'catalina', 'o', 'ladrón', 'catalina', 'catalina', 'catalina', 'correrías', 'catalina', 'o', 'ca

## Finally just a few functions to organize the data: get rid of duplicates and organize by frequency and then write all that to a text file

In [131]:
# convert filtered words list to panda data frame
filtered_df = pd.DataFrame(filtered_words, columns=['word'])

# get rid of duplicates
filtered_df.drop_duplicates(inplace=True)

# order by freq
filtered_df = filtered_df.merge(df, how = 'left', left_on = 'word', right_on = 'lemma')
filtered_df.sort_values(by = 'freq', ascending = True, inplace = True)

# write to a text file
output_file = "B2_word_list.txt"
with open(output_file, "w", encoding = "utf-8") as file:
    for word in zip(filtered_df['word']):
        file.write(f"{word}\n")
